주차수요예측경진대회

# STEP1. 패키지 및 데이터 로딩

In [1]:
import os 
import glob
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

import plotly 
import plotly.express as px
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# plotly 그래프 업로드
pd.options.plotting.backend = 'plotly'
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

# 옵션 설정
pd.set_option('display.max_columns', None)
rc('font', family='Malgun Gothic')
matplotlib.rcParams['axes.unicode_minus'] = False

In [3]:
current_directory = os.getcwd()
print(current_directory) 

C:\Users\youth_0619\OneDrive - 중앙대학교\바탕 화면


In [4]:
train = pd.read_csv("C:/Users/youth_0619/OneDrive - 중앙대학교/바탕 화면/대학/학회 및 동아리/CUAI/dacon/train.csv")
test = pd.read_csv("C:/Users/youth_0619/OneDrive - 중앙대학교/바탕 화면/대학/학회 및 동아리/CUAI/dacon/test.csv")
submission = pd.read_csv("C:/Users/youth_0619/OneDrive - 중앙대학교/바탕 화면/대학/학회 및 동아리/CUAI/dacon/sample_submission.csv")

# STEP2. 데이터 탐색

In [5]:
train.shape, test.shape

((2952, 15), (1022, 14))

In [6]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [7]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


## 1) 컬럼명 변경  

In [8]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', 
    '전용면적', '전용면적별세대수', '공가수', '자격유형', '임대보증금',
    '임대료', '지하철', '버스', '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', 
    '전용면적', '전용면적별세대수', '공가수', '자격유형', '임대보증금', 
    '임대료', '지하철', '버스', '단지내주차면수'
]

##  2) 데이터 타입 및 결측치 확인

In [9]:
train.dtypes, train.isnull().sum()

(단지코드         object
 총세대수          int64
 임대건물구분       object
 지역           object
 공급유형         object
 전용면적        float64
 전용면적별세대수      int64
 공가수         float64
 자격유형         object
 임대보증금        object
 임대료          object
 지하철         float64
 버스          float64
 단지내주차면수     float64
 등록차량수       float64
 dtype: object,
 단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 자격유형          0
 임대보증금       569
 임대료         569
 지하철         211
 버스            4
 단지내주차면수       0
 등록차량수         0
 dtype: int64)

In [10]:
test.dtypes, test.isnull().sum()

(단지코드         object
 총세대수          int64
 임대건물구분       object
 지역           object
 공급유형         object
 전용면적        float64
 전용면적별세대수      int64
 공가수         float64
 자격유형         object
 임대보증금        object
 임대료          object
 지하철         float64
 버스          float64
 단지내주차면수     float64
 dtype: object,
 단지코드          0
 총세대수          0
 임대건물구분        0
 지역            0
 공급유형          0
 전용면적          0
 전용면적별세대수      0
 공가수           0
 자격유형          2
 임대보증금       180
 임대료         180
 지하철          42
 버스            0
 단지내주차면수       0
 dtype: int64)

# STEP3. 전처리

In [11]:
train[['지하철', '버스']] = train[['지하철', '버스']].fillna('0').astype(int)
test[['지하철']] = test[['지하철']].fillna('0'). astype(int)

In [12]:
test[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0,2.0,173.0


In [13]:
test[(test.단지코드=='C2411') | (test.단지코드=='C2253')].head(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,NaN,NaN,0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,NaN,NaN,0,2.0,173.0


In [14]:
test.loc[(test.단지코드=='C2411')&(test.자격유형.isnull()), '자격유형'] = 'A'
test.loc[(test.단지코드=='C2253')&(test.자격유형.isnull()), '자격유형'] = 'C'

test[(test.단지코드=='C2411') | (test.단지코드=='C2253')].head(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,A,71950000,37470,0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,C,2249000,44770,0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,NaN,NaN,0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,NaN,NaN,0,2.0,173.0


In [15]:
# 전용면적 16.91 
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 16.91), '임대보증금'] = 13450000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 16.91), '임대료'] = 65500
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 19700000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 96000
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 19150000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 94000
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 21400000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 105000

In [16]:
# 전용면적 24.72
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.72), '임대보증금'] = 7000000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.72), '임대료'] = 135000
# 전용면적 24.79
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.79), '임대보증금'] = 7000000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.79), '임대료'] = 135000
# 전용면적 26.83
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 26.83), '임대보증금'] = 7600000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 26.83), '임대료'] = 142000
# 전용면적 37.7
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 37.7), '임대보증금'] = 14800000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 37.7), '임대료'] = 198000
# 전용면적 46.94
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 46.94), '임대보증금'] = 23100000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 46.94), '임대료'] = 259000

In [17]:
# 전용면적 29.17
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.17), '임대보증금'] = 10847000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.17), '임대료'] = 138600
# 전용면적 29.34
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.34), '임대보증금'] = 10847000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.34), '임대료'] = 138600
# 전용면적 37.43
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 37.43), '임대보증금'] = 17338000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 37.43), '임대료'] = 197500

In [18]:
# 전용면적 24.83
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 24.83), '임대보증금'] = 2129000
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 24.83), '임대료'] = 42350
# 전용면적 33.84
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 33.84), '임대보증금'] = 2902000
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 33.84), '임대료'] = 57730

In [19]:
# 전용면적 16.94
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 16.94), '임대보증금'] = 11200000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 16.94), '임대료'] = 53200
# 전용면적 26.85
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대보증금'] = 16333330
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대료'] = 77580
# 전용면적 26.85
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대보증금'] = 18620000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대료'] = 88440
# 전용면적 36.77
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 36.77), '임대보증금'] = 23760000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 36.77), '임대료'] = 112860

In [20]:
# 전용면적 26.37
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '임대보증금'] = 5787000
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '임대료'] = 79980
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '자격유형'] = 'C'
# 전용면적 52.74
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '임대보증금'] = 11574000
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '임대료'] = 159960
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '자격유형'] = 'C'

In [21]:
train[['임대보증금','임대료']] = train[['임대보증금','임대료']].fillna('0').replace('-','0').astype(int)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna('0').replace('-','0').astype(int)

In [22]:
train.dtypes, train.isnull().sum()

(단지코드         object
 총세대수          int64
 임대건물구분       object
 지역           object
 공급유형         object
 전용면적        float64
 전용면적별세대수      int64
 공가수         float64
 자격유형         object
 임대보증금         int32
 임대료           int32
 지하철           int32
 버스            int32
 단지내주차면수     float64
 등록차량수       float64
 dtype: object,
 단지코드        0
 총세대수        0
 임대건물구분      0
 지역          0
 공급유형        0
 전용면적        0
 전용면적별세대수    0
 공가수         0
 자격유형        0
 임대보증금       0
 임대료         0
 지하철         0
 버스          0
 단지내주차면수     0
 등록차량수       0
 dtype: int64)

In [23]:
test.dtypes, test.isnull().sum()

(단지코드         object
 총세대수          int64
 임대건물구분       object
 지역           object
 공급유형         object
 전용면적        float64
 전용면적별세대수      int64
 공가수         float64
 자격유형         object
 임대보증금         int32
 임대료           int32
 지하철           int32
 버스          float64
 단지내주차면수     float64
 dtype: object,
 단지코드        0
 총세대수        0
 임대건물구분      0
 지역          0
 공급유형        0
 전용면적        0
 전용면적별세대수    0
 공가수         0
 자격유형        0
 임대보증금       0
 임대료         0
 지하철         0
 버스          0
 단지내주차면수     0
 dtype: int64)

## 3) 오류 데이터 제거

In [24]:
error_data = ['C2085', 'C1397', 'C2431', 'C1649', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']

for error in error_data:
    error_index = train[train['단지코드'] == error].index
    train.drop(error_index, inplace=True)

In [25]:
train.shape, test.shape

((2876, 15), (1022, 14))

## 4) 파생변수 생성

In [26]:
# 지역별 샘플수(count)와 등록차량수의 중앙값(median) 산출 후 중앙값 기준으로 정렬
area_groups = pd.DataFrame([
    *pd.DataFrame({
        '지역' : train['지역'],
        '등록차량수' : train['등록차량수'],
    })
    .groupby(['지역'])
    .apply(lambda x: {
        '지역' : x.iloc[0,0],
        'count' : len(x),
        'median' : x.등록차량수.median()
    })
]).sort_values('median')

# 중앙값이 작은 지역부터 누적 샘플크기(cum_count)를 토대로 백분위수를 고려해 5개 그룹으로 분할
area_groups['cum_count'] = np.cumsum(area_groups['count'])
area_groups['n지역'] = pd.qcut(area_groups['cum_count'], 5, labels=False, retbins=False)

to_join = area_groups[['지역', 'n지역']].set_index('지역')
train = train.join(to_join, on='지역')
train['n지역'] = train['n지역'].astype('category')

In [27]:
train = train[['단지코드', '총세대수', '임대건물구분', '지역', 'n지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '등록차량수']]

In [28]:
test.insert(4, 'n지역','')
test.loc[(test['지역'] == '대전광역시') | (test['지역'] == '충청남도') | (test['지역'] == '부산광역시') | (test['지역'] == '제주특별자치도'), 'n지역'] = 0
test.loc[(test['지역'] == '강원도') | (test['지역'] == '경상남도') | (test['지역'] == '울산광역시'), 'n지역'] = 1
test.loc[(test['지역'] == '전라남도') | (test['지역'] == '전라북도') | (test['지역'] == '충청북도'), 'n지역'] = 2
test.loc[(test['지역'] == '경상북도') | (test['지역'] == '광주광역시') | (test['지역'] == '서울특별시'), 'n지역'] = 3
test.loc[(test['지역'] == '경기도') | (test['지역'] == '대구광역시') | (test['지역'] == '세종특별자치시'), 'n지역'] = 4

test['n지역'] = test['n지역'].astype('category')

In [29]:
px.box(train, x='n지역', y='등록차량수', hover_name='단지코드')

In [32]:
train['n공급유형'] = train.loc[:,'공급유형'].astype('category').cat.codes
test['n공급유형'] = test.loc[:,'공급유형'].astype('category').cat.codes

In [33]:
train['전용면적'] = train['전용면적'] //5*5
train['전용면적'] = np.where(train['전용면적'] > 100, 100, train['전용면적'])
train['전용면적'] = np.where(train['전용면적'] < 15, 15, train['전용면적'])


test['전용면적'] = test['전용면적'] //5*5
test['전용면적'] = np.where(test['전용면적'] > 100, 100, test['전용면적'])
test['전용면적'] = np.where(test['전용면적'] < 15, 15, test['전용면적'])

In [34]:
codes = train.단지코드.unique()
areas = np.sort(train.전용면적.unique())

df_train = pd.DataFrame()
columns = ['단지코드', 'n지역', 'n공급유형', '공가수', '지하철', '버스', '단지내주차면수', '등록차량수']

for i, code in enumerate(codes):
    tmp_code = train.loc[train.단지코드 == code].reset_index(drop=True)

    df_train.loc[i, columns] = tmp_code.loc[0, columns]
    df_train.loc[i, '총임대세대수'] = (tmp_code.전용면적별세대수).sum()

    df_train.loc[i, '평균전용면적'] = (tmp_code.전용면적).mean() 
    df_train.loc[i, '총전용면적'] = (tmp_code.전용면적 * tmp_code.전용면적별세대수).sum()
    df_train.loc[i, '평균임대보증금'] = (tmp_code.전용면적별세대수 * tmp_code.임대보증금).sum() / (tmp_code.전용면적별세대수).sum()
    df_train.loc[i, '평균임대료'] = (tmp_code.전용면적별세대수 * tmp_code.임대료).sum() / (tmp_code.전용면적별세대수).sum()

In [35]:
codes = test.단지코드.unique()
areas = np.sort(test.전용면적.unique())

df_test = pd.DataFrame()
columns = ['단지코드', 'n지역', 'n공급유형', '공가수', '지하철', '버스', '단지내주차면수']

for i, code in enumerate(codes):
    tmp_code = test.loc[test.단지코드 == code].reset_index(drop=True)

    df_test.loc[i, columns] = tmp_code.loc[0, columns]
    df_test.loc[i, '총임대세대수'] = (tmp_code.전용면적별세대수).sum()

    df_test.loc[i, '평균전용면적'] = (tmp_code.전용면적).mean() 
    df_test.loc[i, '총전용면적'] = (tmp_code.전용면적 * tmp_code.전용면적별세대수).sum()
    df_test.loc[i, '평균임대보증금'] = (tmp_code.전용면적별세대수 * tmp_code.임대보증금).sum() / (tmp_code.전용면적별세대수).sum()
    df_test.loc[i, '평균임대료'] = (tmp_code.전용면적별세대수 * tmp_code.임대료).sum() / (tmp_code.전용면적별세대수).sum()

# STEP4. 모형 학습, 예측 및 평가

#### 선형 회귀(Linear Regression)

In [37]:
y_target = df_train['등록차량수']
X_data = df_train.drop(['단지코드', '등록차량수'], axis=1, inplace=False) 

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, shuffle=False, random_state=42, test_size=0.1)

In [38]:
X_test.shape

(42, 11)

In [39]:
model = LinearRegression()
model.fit(X_train, y_train)

print(f'R2 Score : {r2_score(model.predict(X_test), y_test)}, MAE : {mean_absolute_error(model.predict(X_test), y_test)}')

R2 Score : 0.8399127757741529, MAE : 122.52178023341351


In [40]:
from sklearn.model_selection import cross_val_score

mae_scores = cross_val_score(model, X_data, y_target, scoring='neg_mean_absolute_error', cv=5)
avg_mae = np.mean(mae_scores)

print(f'5Fold 개별 MAE score : ', mae_scores)
print(f'5Fold 평균 MAE : ', avg_mae)

5Fold 개별 MAE score :  [-211.37859138 -112.72367756  -90.65526517 -127.62400448 -166.17381462]
5Fold 평균 MAE :  -141.71107064358566


In [41]:
df_test.head()

,단지코드,n지역,n공급유형,공가수,지하철,버스,단지내주차면수,총임대세대수,평균전용면적,총전용면적,평균임대보증금,평균임대료
0,C1072,4.0,3.0,14.0,0.0,2.0,683.0,754.0,46.250000,34650.0,3.772908e+07,264048.249337
1,C1128,4.0,3.0,9.0,0.0,3.0,1216.0,1354.0,43.333333,59620.0,3.512774e+07,251290.044313
2,C1456,0.0,3.0,18.0,0.0,16.0,547.0,619.0,41.666667,26075.0,3.422472e+07,224706.187399
3,C1840,2.0,3.0,7.0,0.0,3.0,543.0,593.0,41.250000,23755.0,1.806555e+07,120287.537943
4,C1332,4.0,3.0,11.0,0.0,2.0,1112.0,1297.0,43.750000,55635.0,4.010183e+07,284193.623747


In [42]:
Xtest = df_test.drop(['단지코드'],axis=1, inplace=False)
pred = model.predict(Xtest)

In [43]:
pred.shape

(150,)

# STEP6. 제출

In [44]:
submission['num'] = pred
submission.to_csv('final_output.csv', index=False)